## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

In [29]:
import mne
data = mne.read_epochs("../../datasets/avi/multi/sub1_combined-raw.fif")

Reading c:\Users\caio-\Documents\facul\patter-recog\SSVEP\avi\classifier\..\..\datasets\avi\multi\sub1_combined-raw.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   15998.05 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


C:\Users\caio-\AppData\Local\Temp\ipykernel_3140\343106749.py:2: RuntimeWarning: This filename (../../datasets/avi/multi/sub1_combined-raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data = mne.read_epochs("../../datasets/avi/multi/sub1_combined-raw.fif")


In [30]:
# TAREFA 2: Início da tarefa 2
print("Tarefa 2")

threshold = 0.23  # Define o valor do limite como 0.23
data
print(f"\n") 

# TAREFA 3: Início da tarefa 3

import numpy as np

targets = [float(item) for item in data.event_id.keys()]  # Cria uma lista de alvos a partir das chaves do dicionário 'event_id' em 'data'
print(targets)  # imprime o tipo de 'targets'

features = list()  # Inicializa uma lista vazia chamada 'features'
for i in range(len(data)):  # Loop for que itera sobre os elementos de 'data'
    sample = list()  # Inicializa uma lista vazia chamada 'sample'
    for target in targets:  # Loop aninhado que itera sobre os alvos
        fmin = target - threshold  # Calcula 'fmin' subtraindo o 'threshold' do alvo
        fmax = target + threshold  # Calcula 'fmax' somando o 'threshold' ao alvo
        sample.append(data[i].compute_psd(method='multitaper', fmin=fmin,
                                          fmax=fmax, verbose=False))  # Calcula o PSD (Power Spectral Density) para o i-ésimo elemento de 'data' no intervalo [fmin, fmax] e adiciona a 'sample'
    features.append(sample)  # Adiciona 'sample' à lista 'features'

arrayFeatures = np.array(features)  # converte 'features' em um array NumPy e armazena em 'arrayFeatures'
arrayFeatures.shape #formato dos dados(array)


Tarefa 2


[6.0, 6.5, 7.0, 7.5, 8.2, 9.3]


(100, 6, 1, 1, 7)

In [31]:
# organizando os dados
arrayFeatures = arrayFeatures.reshape(arrayFeatures.shape[0], arrayFeatures.shape[1], arrayFeatures.shape[-1])
print(arrayFeatures.shape) #formato pós padronização PSD

# TAREFA 4

#max values do array
maxValues = np.max(arrayFeatures, axis=-1)
maxValues.shape

(100, 6, 7)


(100, 6)

In [32]:
# TAREFA 5

# Carrega os rótulos (labels)
labels = np.load("../../datasets/avi/multi/labelsMulti_sub1.npy")

# Realiza uma verificação dos dados do rótulo carregados, exibindo os rótulos, sua forma (shape) e os targets (alvos)
# print(labels, labels.shape, targets)

# Classificação pelo maior valor
# Encontra o índice do maior valor em maxValues e armazena em iMax
iMax = maxValues.argmax(axis=-1)

# Cria uma lista de '1' para cada índice onde o target correspondente é igual ao rótulo em iMax
hits = [1 for i in range(len(iMax)) if targets[iMax[i]] == labels[i]]

# Calcula a acurácia como a porcentagem de acertos em relação ao total de rótulos
acc = 100 * sum(hits) / len(labels)

print(f'Accuracy: {acc:.2f}%')


Accuracy: 78.00%
